In [5]:
import os
import pandas as pd
import shutil   #for copying the files
from sklearn.model_selection import train_test_split

# Original dataset folder
original_folder = r'original_dataset'  

# New folders for Kaggle test set and the remaining data
kaggle_test_folder = r'kaggle_solutionSet'
remaining_data_folder = r'competition_data'

# Create new directories
os.makedirs(kaggle_test_folder, exist_ok=True)
os.makedirs(remaining_data_folder, exist_ok=True)

# CSV file to hold the solution for Kaggle scoring
solution_csv = []
solution_csv_path = 'solution.csv'

# Check current working directory
print("Current Working Directory:", os.getcwd())

# Check if 'original_folder' exists
if os.path.exists(original_folder):
    print(f"The folder {original_folder} exists.")
else:
    print(f"The folder {original_folder} does not exist.")

# Loop through each sub-folder (i.e., each class) in the original dataset
for label in os.listdir(original_folder):
    src_path = os.path.join(original_folder, label)

    # Create a destination folder for each class in the Kaggle test folder and remaining data folder
    dest_path_test = os.path.join(kaggle_test_folder, label)
    dest_path_remaining = os.path.join(remaining_data_folder, label)

    os.makedirs(dest_path_test, exist_ok=True)
    os.makedirs(dest_path_remaining, exist_ok=True)

    # Get all files in the source folder
    files = [f for f in os.listdir(src_path) if os.path.isfile(os.path.join(src_path, f))]
    labels = [label] * len(files)

    # Stratified 80-20 split
    remaining_files, test_files = train_test_split(files, test_size=0.2, random_state=42, stratify=labels)

    # Copy the files
    for f in remaining_files:
        shutil.copy(os.path.join(src_path, f), os.path.join(dest_path_remaining, f))

    for f in test_files:
        shutil.copy(os.path.join(src_path, f), os.path.join(dest_path_test, f))
        solution_csv.append({'ID': f, 'Class': label, 'Usage': 'Private'})

# Create the solution CSV file for Kaggle scoring
df = pd.DataFrame(solution_csv)
df.to_csv(solution_csv_path, index=False)


Current Working Directory: f:\Grad_School\CSCE_5215\GroupProject\Git\BrainSynergy\data
The folder original_dataset exists.
